# Intro to TensorFlow

Vincent Vanhoucke, Principal Scientist at Google Brain, introduces you to deep learning and Tensorflow, Google's deep learning framework.

## Hello, Tensor World!

In [1]:
import tensorflow as tf

# Values are encapsulated in an object called a tensor
hello_constant = tf.constant("Hello, World!")
int_constant   = tf.constant(1234)
array_constant = tf.constant([[10,20], [30,40]])

# Session is an environment for running computational graph built with TF API
with tf.Session() as sess:
    output = sess.run(hello_constant)
    print(output)

b'Hello, World!'


## Input

In [2]:
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)

with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Hello, World!', y: 20})

## Math

In [3]:
# z = x/y - 10
x = tf.placeholder(tf.int32)
y = tf.placeholder(tf.int32)
z = tf.subtract(tf.cast(tf.divide(x,y), tf.int32), tf.constant(10))

with tf.Session() as sess:
    output = sess.run(z, feed_dict={x: 60, y: 5})
    print(output)

2


## Linear Classifier for MNIST dataset

### Basics Linear Function

In [4]:
# Variables to represent weights and bias
x = tf.Variable(5)
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
# Generate random numbers from a normal distribution for weights
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

# Generate zeros
bias = tf.Variable(tf.zeros(n_labels))

### Basic Implementation

In [5]:
from tensorflow.examples.tutorials.mnist import input_data

# Prepare MNIST dataset
def mnist_features_labels(n_labels):
    '''
    Get the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    '''
    mnist_features = []
    mnist_labels = []
    
    mnist = input_data.read_data_sets('tmp', one_hot=True)
    
    # only looking at first 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):
        # only looking at up to <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])
            
    return mnist_features, mnist_labels

n_features = 28 * 28
n_labels = 3

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.float32)

train_features, train_labels = mnist_features_labels(n_labels)

# Initialise Variables and Linear Function
w = tf.Variable(tf.truncated_normal((n_features, n_labels)))
b = tf.Variable(tf.zeros(n_labels))

logits = tf.add(tf.matmul(features, w), b)

# Hyperparameters
learning_rate = 0.08

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    # Softmax Activation Function
    prediction = tf.nn.softmax(logits)
    
    # Cross Entropy Cost Function
    cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
    
    # Training Loss
    loss = tf.reduce_mean(cross_entropy)
    
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Run optimizer and get loss
    _, l = sess.run(
            [optimizer, loss],
            feed_dict={features: train_features, labels: train_labels})
    
print('Loss: {}'.format(l))

Extracting mnist_tmp/train-images-idx3-ubyte.gz
Extracting mnist_tmp/train-labels-idx1-ubyte.gz
Extracting mnist_tmp/t10k-images-idx3-ubyte.gz
Extracting mnist_tmp/t10k-labels-idx1-ubyte.gz
Loss: 3.845045328140259


###  Softmax Activation Function

In [6]:
logit_data = [2.0, 1.0, 0.1]
logits = tf.placeholder(tf.float32)

softmax = tf.nn.softmax(logits)

with tf.Session() as sess:
    output = sess.run(softmax, feed_dict={logits: logit_data})
    print(output)

[ 0.65900117  0.24243298  0.09856589]


### One-Hot Encoding with Scikit-Learn

In [7]:
from sklearn import preprocessing
import numpy as np

labels = np.array([1,5,3,2,1,4,2,1,3])

lb = preprocessing.LabelBinarizer()
lb.fit(labels)
lb.transform(labels)

array([[1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0]])

### Cross Entropy Cost Function

In [8]:
softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

with tf.Session() as sess:
    print(sess.run(cross_entropy, feed_dict={softmax: softmax_data, one_hot: one_hot_data}))

0.356675


## Numerical Stability

In [9]:
a = 1000000000
for i in range(1000000):
    a = a + 1e-6
print(a - 1000000000)

0.95367431640625


## Mini-batch and Epochs

In [10]:
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    out_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        out_batches.append(batch)
    
    return out_batches

In [11]:
# Hyperparameters
learning_rate = 0.1
batch_size = 128
epochs = 10

# Prepare MNIST dataset
n_input = 28 * 28
n_classes = 10

mnist = input_data.read_data_sets('mnist_tmp', one_hot=True)

train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Initialise Variables
weights = tf.Variable(tf.random_normal([n_input, n_classes]))
bias    = tf.Variable(tf.random_normal([n_classes]))

logits  = tf.add(tf.matmul(features, weights), bias)
learn   = tf.placeholder(tf.float32)
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
gdo     = tf.train.GradientDescentOptimizer(learning_rate=learn).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    train_batches = batches(batch_size, train_features, train_labels)

    # Epoch Loop
    for epoch_i in range(epochs):
        #Batch Loop
        for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
            sess.run(gdo, feed_dict={features: batch_features, labels: batch_labels, learn: learning_rate})
        
        current_cost = sess.run(cost, feed_dict={features: batch_features, labels: batch_labels})
        valid_accuracy = sess.run(accuracy, feed_dict={features: valid_features, labels: valid_labels})
        print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
            epoch_i,
            current_cost,
            valid_accuracy
            ))
    
    test_accuracy = sess.run(accuracy, feed_dict={features: test_features, labels: test_labels})
    
print('Test Accuracy: {}\n'.format(test_accuracy))

Extracting mnist_tmp/train-images-idx3-ubyte.gz
Extracting mnist_tmp/train-labels-idx1-ubyte.gz
Extracting mnist_tmp/t10k-images-idx3-ubyte.gz
Extracting mnist_tmp/t10k-labels-idx1-ubyte.gz
Epoch: 0    - Cost: 1.64     Valid Accuracy: 0.73 
Epoch: 1    - Cost: 1.13     Valid Accuracy: 0.801
Epoch: 2    - Cost: 0.96     Valid Accuracy: 0.828
Epoch: 3    - Cost: 0.867    Valid Accuracy: 0.845
Epoch: 4    - Cost: 0.805    Valid Accuracy: 0.856
Epoch: 5    - Cost: 0.759    Valid Accuracy: 0.866
Epoch: 6    - Cost: 0.722    Valid Accuracy: 0.872
Epoch: 7    - Cost: 0.691    Valid Accuracy: 0.877
Epoch: 8    - Cost: 0.665    Valid Accuracy: 0.88 
Epoch: 9    - Cost: 0.642    Valid Accuracy: 0.883
Test Accuracy: 0.873199999332428

